In [1]:
import pandas as pd

url = "https://docs.google.com/spreadsheets/d/e/2PACX-1vQ2G-ZFSotS2qo94mnTWCDvj9Y49ai-9O61DA7940sPYynEdBXq2cT2-Wl3nNldIb3gkpbPFaTFY2PJ/pub?output=xlsx"

xls = pd.ExcelFile(url)
xls

In [2]:
sheets = {
    sheet_name: pd.read_excel(xls, sheet_name)
    for sheet_name in xls.sheet_names
}
for sheet in sheets.values():
    sheet['grounding'] = sheet['grounding'].map(eval)
set(sheets)

{'BIOMD0000000955',
 'BIOMD0000000956',
 'BIOMD0000000957',
 'BIOMD0000000958',
 'BIOMD0000000960'}

In [5]:
from collections import defaultdict
ground_truth_df = pd.read_csv("../mira/resources/mapped_biomodels_groundings.csv", sep=',')
dd = defaultdict(set)
for row_index,entry in ground_truth_df.iterrows():
    model_id = entry["model"]
    if not pd.isna(entry["identifiers"]):
        curie = entry["identifiers"].split("/")[0]
    else:
        curie = entry["mapped_identifiers"].split("/")[0]

    dd[model_id].add(curie)
dd = dict(dd)

In [6]:
rows = []
for model_id, sheet in sheets.items():
    sheet_curies = {
        grounding["id"]
        for groundings in sheet["grounding"]
        for grounding in groundings
    }
    ground_truth_curies = dd[model_id]

    intersection_curies = sheet_curies & ground_truth_curies
    n_intersection = len(intersection_curies)
    n_ground_truth = len(ground_truth_curies)
    n_sheet = len(sheet_curies)
    rows.append((
        model_id,
        n_ground_truth,
        n_intersection,
        round(n_intersection / n_ground_truth, 2)
    ))

print("The way to interpret the precision is the percent")
pd.DataFrame(rows, columns=["model", "structured", "TA-1", "precision"])

The way to interpret the precision is the percent


,model,structured,TA-1,precision
0,BIOMD0000000955,4,0,0.00
1,BIOMD0000000956,3,0,0.00
2,BIOMD0000000957,4,1,0.25
3,BIOMD0000000958,6,1,0.17
4,BIOMD0000000960,4,0,0.00
